In [ ]:
#---#| default_exp pipeline_api

# pipeline API

This module provides high-level APIs for different pipelines such as transfer learning, library geneartion, and rescoring (percolator). 

In [ ]:
from peptdeep.pipeline_api import *

## Transfer learning pipeline

Transfer learning pipeline includes:
1. Loading PSM files of the search engine results.
2. Matching PSMs against the MS files.
3. Loading pre-trained models and refining RT/CCS(/MS2) models.

The refined models will be saved in the path pointed by "PEPTDEEP_HOME" in `peptdeep.settings.global_settings`.